## Load Robot

In [2]:
# Import required packages
import gymnasium as gym
import torch
import mani_skill.envs
from tqdm.notebook import tqdm
from mani_skill.utils.wrappers import RecordEpisode
from IPython.display import Video

In [3]:
env = gym.make(
    "Empty-v1",
    obs_mode="none",
    reward_mode="none",
    enable_shadow=True,
    control_mode="pd_joint_pos", # allows us to easily hold joints mostly in place
    robot_uids="unitree_go2",
    render_mode="rgb_array",
)
env = RecordEpisode(env, "./videos", save_trajectory=False)
print(env.agent.keyframes.keys())

/home/Morphabledrone/miniconda3/envs/mani/lib/python3.10/site-packages/torch/random.py:183: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICES= or devices=[]; if you are using device 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(message)


dict_keys(['standing'])


/home/Morphabledrone/miniconda3/envs/mani/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.max_episode_steps to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.max_episode_steps` for environment variables or `env.get_wrapper_attr('max_episode_steps')` that will search the reminding wrappers.
  logger.warn(
/home/Morphabledrone/miniconda3/envs/mani/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.agent to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.agent` for environment variables or `env.get_wrapper_attr('agent')` that will search the reminding wrappers.
  logger.warn(


In [ ]:
import sapien
env.reset()
kf = env.agent.keyframes["standing"]
env.agent.robot.set_pose(sapien.Pose([0, 0, 2]))
env.agent.robot.set_qpos(kf.qpos)

# if running on a GPU environment you have to
# apply the changes to environment state
if env.gpu_sim_enabled:
    env.scene._gpu_apply_all() # applies changes
    env.scene.px.gpu_update_articulation_kinematics() # updates robot link poses necessary to render correctly
    env.scene._gpu_fetch_all() # updates GPU buffers
env.render() # call this to update the rendering
for i in tqdm(range(100)):
    # with pd_joint_pos control we try to get the robot to maintain the same joint
    # positions as the keyframe
    obs, reward, terminated, truncated, info = env.step(kf.qpos)
env.flush_video("example")
Video("./videos/example.mp4", embed=True, width=640) # Watch our replay

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
from mani_skill.agents.registration import REGISTERED_AGENTS
REGISTERED_AGENTS.keys()

dict_keys(['allegro_hand_right', 'allegro_hand_left', 'allegro_hand_right_touch', 'anymal_c', 'dclaw', 'fetch', 'floating_panda_gripper', 'floating_robotiq_2f_85_gripper', 'googlerobot', 'humanoid', 'panda', 'panda_wristcam', 'panda_stick', 'stompy', 'trifingerpro', 'unitree_g1', 'unitree_g1_simplified_legs', 'unitree_g1_simplified_upper_body', 'unitree_g1_simplified_upper_body_right_arm', 'unitree_go2', 'unitree_go2_simplified_locomotion', 'unitree_h1', 'unitree_h1_simplified', 'ur_10e', 'widowx250s', 'xarm7_ability', 'xmate3_robotiq'])